### Imports

In [ ]:
% matplotlib inline

import os
import sys

sys.path.append('../..')
os.chdir('../..')

from io import BytesIO
from pathlib import Path

import menpo.io as mio
import tensorflow as tf
from menpo.visualize import print_progress

from menpowidgets import visualize_images
from project.utils import tfrecords

# Generate TFRecords files

The recommended format for TensorFlow is a TFRecords file containing `tf.train.Example` protocol buffers (which contain `Features` as a field).

Here is a little program that gets your data, stuffs it in an `Example` protocol buffer, serializes the protocol buffer to a string, and then writes the string to a TFRecords file using the `tf.python_io.TFRecordWriter`.

In [ ]:
def face_iterator(images):
    """ Given an iterable of images, returns a generator of cat face data """
    for idx, img in enumerate(print_progress(images, end_with_newline=False)):
        image_name = 'frame_{}'.format(idx)

        yield image_name, img


def generate(iterator,
             store_path='./',
             record_name='inference.tfrecords',
             store_records=True):
    store_path = Path(store_path)

    if store_records:
        writer = tf.python_io.TFRecordWriter(str(store_path / record_name))

    for img_name, pimg in iterator:
        
        # resize image to 256 * 256
        cimg = pimg.resize([256, 256])

        img_path = store_path / '{}'.format(img_name)

        if store_records:
            try:
                # construct the Example proto object
                example = tf.train.Example(
                    features=tf.train.Features(
                        # Features contains a map of string to Feature proto objects
                        feature={
                            # images
                            'image': tfrecords.jpg_feature(cimg),
                            'height': tfrecords.int_feature(cimg.shape[0]),
                            'width': tfrecords.int_feature(cimg.shape[1]),
                        }))
                # use the proto object to serialize the example to a string
                serialized = example.SerializeToString()
                # write the serialized object to disk
                writer.write(serialized)

            except Exception as e:
                print('Something bad happened when processing image: "{}"'.format(img_name))
                print(e)

    if store_records:
        writer.close()

## Generate TFRecords for Inference

Run the following cells to generate TFRecords for a directory of images to perform inference on

In [ ]:
# Where are the images located?
video_dir = Path('D:/dev/project/data/ibug/cats/videos')
video_name = 'video6.mp4'

video_path = video_dir / video_name

# where should the resulting TFRecords files be written to?
store_path = Path('data/tfrecords/inference')
inference_record_name = Path(video_path.stem + '.tfrecords')

In [ ]:
# Run this to generate the TFRecord file!

# load the images
images = mio.import_video(video_path)

print('Found {} assets'.format(len(images)))

# generate TFRecords
generate(face_iterator(images), store_path, inference_record_name,
         store_records=True)

print('Created {}'.format(store_path / inference_record_name))

In [ ]:
visualize_images(images, browser_style ='slider')